### <center>Signal Processing</center>

**Note**: _This file is only for testing purposes. It is not user-friendly and requires user to adjust it for themselves._


In [ ]:
%pip install -r "D:\\natHacks\\NeuroKnights\\requirements.txt"

#### Imports


In [1]:
import time
import serial.tools.list_ports

from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds

#### Listing available ports


In [2]:
ports = serial.tools.list_ports.comports()
print([port.device for port in ports])

['COM6', 'COM3']


In [3]:
BoardShim.enable_dev_board_logger()

params = BrainFlowInputParams()

params.serial_port = "COM6"

board = BoardShim(BoardIds.NEUROPAWN_KNIGHT_BOARD, params)

In [ ]:
print(board.get_sampling_rate(BoardIds.NEUROPAWN_KNIGHT_BOARD))
# print(board.get_eeg_channels(-1))

In [4]:
board.prepare_session()
print(board.is_prepared())

True


In [5]:
board.start_stream(450000)

In [ ]:
print(
    f"Indecies for EEG channels {BoardShim.get_eeg_channels(BoardIds.NEUROPAWN_KNIGHT_BOARD)}"
)
print(
    f"Indecies for EXG channels {BoardShim.get_exg_channels(BoardIds.NEUROPAWN_KNIGHT_BOARD)}"
)

In [6]:
channels = list(range(1, 5))  # TODO: change to using get_exg_channels
gain_value = 12

In [7]:
for channel in channels:
    board.config_board(f"chon_{channel}_{gain_value}")
    print(f"Enabled channel {channel} with gain {gain_value}.")
    time.sleep(0.5)

Enabled channel 1 with gain 12.
Enabled channel 2 with gain 12.
Enabled channel 3 with gain 12.
Enabled channel 4 with gain 12.


In [10]:
assert board
data = board.get_current_board_data(1000)
data

array([[ 2.30000000e+02,  2.31000000e+02,  2.32000000e+02, ...,
         2.03000000e+02,  2.04000000e+02,  2.05000000e+02],
       [ 5.94377563e+02, -2.99731934e+02,  1.81158401e+02, ...,
         6.00258606e+02, -2.98301422e+02,  4.83354004e+02],
       [ 3.32752899e+02, -3.14076782e+02,  1.40190140e+02, ...,
         4.15086792e+02, -3.76621918e+02,  3.91165466e+02],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.73170682e+09,  1.73170682e+09,  1.73170682e+09, ...,
         1.73170683e+09,  1.73170683e+09,  1.73170683e+09],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [13]:
data.shape

(13, 1000)

In [18]:
import numpy as np

In [26]:
eeg = data[1:5].T
eeg

array([[ 594.37756348,  332.75289917,  471.51254272,  323.41485596],
       [-299.73193359, -314.07678223, -372.21118164, -340.34255981],
       [ 181.15840149,  140.19013977,  276.36691284,  232.61709595],
       ...,
       [ 600.25860596,  415.08679199,  542.32287598,  447.03488159],
       [-298.30142212, -376.62191772, -422.00094604, -356.67422485],
       [ 483.35400391,  391.16546631,  537.15710449,  487.48657227]])

In [30]:
eeg.shape

(1000, 4)

In [ ]:
timestamps = np.array(data[11].T)
timestamps.shape = (timestamps.shape[0], 1)
print(timestamps)

[[1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.73170682e+09 1.73170682e+09 1.73170682e+09 1.73170682e+09
  1.7317

In [39]:
timestamps.shape

(1000, 1)

In [38]:
timestamps.shape = (1000, 1)

In [37]:
np.ndim(timestamps)

1

In [36]:
np.concatenate((eeg, timestamps), axis=1)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

#### Attempting to implement analysis

**Recreating the method described in the study**: [Classification of SSVEP signals using the combined FoCCA-KNN method and comparison with other machine learning methods](https://www.sciencedirect.com/science/article/abs/pii/S1746809423003907)

**Electrode placement**: "OZ", "O1", "O2"


In [ ]:
from brainflow.data_filter import DataFilter, FilterTypes

##### Band-stop and band-pass filters


In [ ]:
sampling_rate = BoardShim.get_sampling_rate(board_id=BoardIds.NEUROPAWN_KNIGHT_BOARD)
DataFilter.perform_bandpass(
    data=data,
    sampling_rate=sampling_rate,
    start_freq=6.0,
    stop_freq=40.0,
    order=2,
    filter_type=FilterTypes.BUTTERWORTH_ZERO_PHASE.value,
    ripple=0,
)
# DataFilter.perform_bandstop(  # is this neccesary? we already have bandpass...
#     data=data,
#     sampling_rate=sampling_rate,
#     start_freq=49.0,
#     stop_freq=51.0,
#     order=2,
#     filter_type=FilterTypes.BUTTERWORTH_ZERO_PHASE.value,
#     ripple=0,
# )
# DataFilter.perform_bandstop(  # is this neccesary? we already have bandpass...
#     data=data,
#     sampling_rate=sampling_rate,
#     start_freq=59.0,
#     stop_freq=61.0,
#     order=2,
#     filter_type=FilterTypes.BUTTERWORTH_ZERO_PHASE.value,
#     ripple=0,
# )

##### Common Average Referencing (CAR) Filter


In [ ]:
data -= data.mean()

##### Selecting the appropriate harmonics, channels and coefficients


#### End session


In [11]:
for channel in channels:
    board.config_board(f"choff_{channel}")
    print(f"Disabled channel {channel} with gain {gain_value}.")
    time.sleep(0.5)

Disabled channel 1 with gain 12.
Disabled channel 2 with gain 12.
Disabled channel 3 with gain 12.
Disabled channel 4 with gain 12.


In [12]:
board.stop_stream()
board.release_session()